In [1]:
using Pkg

Pkg.activate("../Project.toml")

using Random
import .Iterators: cycle, Cycle, take

using JLD2
using CUDA
using Statistics
using Random

import Base: length, size, iterate, eltype, IteratorSize, IteratorEltype, haslength, @propagate_inbounds, repeat, rand, tail
using Plots; default(fmt=:png,ls=:auto)

import Knet
using Knet: deconv4, conv4, unpool, pool, mat, sigm, KnetArray, nll, zeroone, progress, adam!, sgd!, param, param0, dropout, relu, minibatch, Data
import Knet: train!
using Knet

 Activating environment at `~/yolo-julia/Project.toml`


In [2]:
include("../coco2014.jl")

od = load_data("../dataset/overfit_data.jld2");


┌ Info: Loaded: ../dataset/overfit_data.jld2.
└ @ Main /home/ec2-user/yolo-julia/coco2014.jl:43


In [3]:
include("../models.jl")

Random.seed!(1)
darknet = Darknet("../yolov3.cfg"; verbose=false);


@diff a = darknet(xx, yy)

# a = darknet(xx, yy)


Stacktrace:
 [1] (::var"#37#38")() at /home/ec2-user/.julia/packages/AutoGrad/TTpeo/src/core.jl:205
 [2] differentiate(::Function; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at /home/ec2-user/.julia/packages/AutoGrad/TTpeo/src/core.jl:144
 [3] differentiate(::Function) at /home/ec2-user/.julia/packages/AutoGrad/TTpeo/src/core.jl:135
 [4] top-level scope at In[3]:7
 [5] include_string(::Function, ::Module, ::String, ::String) at ./loading.jl:1091
 [6] execute_code(::String, ::String) at /home/ec2-user/.julia/packages/IJulia/rWZ9e/src/execute_request.jl:27
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/ec2-user/.julia/packages/IJulia/rWZ9e/src/execute_request.jl:86
 [8] #invokelatest#1 at ./essentials.jl:710 [inlined]
 [9] invokelatest at ./essentials.jl:709 [inlined]
 [10] eventloop(::ZMQ.Socket) at /home/ec2-user/.julia/packages/IJulia/rWZ9e/src/eventloop.jl:8
 [11] (::IJulia.var"#15#18")() at ./task.jl:356


LoadError: UndefVarError: xx not defined

In [8]:
dtrn = minibatch(od.x[:, :, :, 1:90], od.y[1:90], 8; xsize = (416,416,3,:), xtype=Knet.atype(), shuffle=true);
dtst = minibatch(od.x[:, :, :, 90:end], od.y[90:end], 2; xsize = (416,416,3,:), xtype=Knet.atype());


In [6]:
function train!(model, train_data::Data, test_data::Data;
                  period::Int=4, iters::Int=100, lr=0.15, optimizer=sgd!)  # or optimizer=adam!
    
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    
    for i in 0:period:iters
        push!(train_loss, model(train_data))
        push!(test_loss, model(test_data))
        
        optimizer(model, take(cycle(train_data), period); lr=lr)
        
        
        println("Iter: ", i, " ", train_loss[end])
    end
    
    return 0:period:iters, train_loss, train_acc, test_loss, test_acc
end

train! (generic function with 2 methods)

In [ ]:
iters, trnloss, trnacc, tstloss, tstacc = train!(
    darknet, dtrn, dtst; 
    period=1, iters=1000, lr=1e-3, optimizer=adam!);


Iter: 0 673.1914366115224
Iter: 1 1074.0528643629768
Iter: 2 5649.146937303976
Iter: 3 22841.72896956097
Iter: 4 37453.229944393854
Iter: 5 16170.971781398599
Iter: 6 6183.348571458078
Iter: 7 2258.7688457404483
Iter: 8 1318.6386725850537
Iter: 9 815.4784805620799
Iter: 10 634.8519304119455
Iter: 11 635.5430593911083
Iter: 12 692.8783674764632
Iter: 13 698.5651530428366
Iter: 14 666.7315221316164
Iter: 15 636.0161181607572
Iter: 16 626.4178831904043
Iter: 17 680.5378372743454
Iter: 18 711.3665289189598
Iter: 19 760.4369940945777
Iter: 20 792.5189533760331
Iter: 21 859.3620715011249
Iter: 22 908.9523521178959
Iter: 23 942.4543584319138
Iter: 24 989.4704275010932
Iter: 25 1042.8589840906316
Iter: 26 1080.9012645453756
Iter: 27 1082.4048702883179
Iter: 28 1136.3681400031123
Iter: 29 1130.7836581491342
Iter: 30 1160.6329879360578
Iter: 31 1126.5884729768743
Iter: 32 1116.7944516864961
Iter: 33 1086.9525245667317
Iter: 34 1066.753478716368
Iter: 35 1040.4395251400767
Iter: 36 1011.552044333

In [ ]:
plot(iters, log.(trnloss), label="train", xlabel="Iterations", ylabel="Loss")
display(plot!(iters, log.(tstloss), label="test"))